# pzfx Library Playground

This notebook demonstrates how to use the pzfx library to read and write GraphPad Prism files.

In [ ]:
# Import the library
from pzfx import read_pzfx, write_pzfx, pzfx_tables
import pandas as pd

## 1. List tables in a .pzfx file

In [ ]:
# List all tables in a test file
tables = pzfx_tables('../tests/testdata/column.pzfx')
print(tables)

## 2. Read a table from a .pzfx file

In [ ]:
# Read by table index (1-based)
df = read_pzfx('../tests/testdata/column.pzfx', table=1)
df

In [ ]:
# Read by table name
df = read_pzfx('../tests/testdata/column.pzfx', table='Data 1')
df

## 3. Read with different strike_action options

- `"exclude"` (default): Replace excluded values with NaN
- `"keep"`: Keep original value
- `"star"`: Append `*` to excluded values

In [ ]:
# Read file with excluded/stricken values
df_exclude = read_pzfx('../tests/testdata/x_y_with_strike.pzfx', strike_action='exclude')
df_keep = read_pzfx('../tests/testdata/x_y_with_strike.pzfx', strike_action='keep')
df_star = read_pzfx('../tests/testdata/x_y_with_strike.pzfx', strike_action='star')

print("strike_action='exclude':")
display(df_exclude)
print("\nstrike_action='keep':")
display(df_keep)
print("\nstrike_action='star':")
display(df_star)

## 4. Write a DataFrame to .pzfx

In [ ]:
# Create a sample DataFrame
df = pd.DataFrame({
    'Control': [1.2, 2.3, 3.1, 2.8],
    'Treatment': [2.5, 3.8, 4.2, 3.9]
})
df

In [ ]:
# Write to .pzfx file
write_pzfx(df, 'my_data.pzfx')
print("Written to my_data.pzfx")

In [ ]:
# Verify by reading it back
df_read = read_pzfx('my_data.pzfx')
df_read

## 5. Write multiple tables

In [ ]:
# Create multiple DataFrames as a dict
data = {
    'Experiment 1': pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]}),
    'Experiment 2': pd.DataFrame({'X': [10, 20], 'Y': [30, 40]})
}

write_pzfx(data, 'multi_table.pzfx')
print("Tables:", pzfx_tables('multi_table.pzfx'))

## 6. Write XY data with X column

In [ ]:
# Create XY data
df_xy = pd.DataFrame({
    'Time': [0, 1, 2, 3, 4],
    'Response': [0.1, 0.5, 0.9, 0.95, 0.99]
})

# Write with Time as X column
write_pzfx(df_xy, 'xy_data.pzfx', x_col='Time')
print("Written XY data")

## 7. Add project notes/metadata

In [ ]:
# Create notes DataFrame with Name/Value columns
# "Notes" is a special name that becomes the project description
notes_df = pd.DataFrame({
    'Name': ['Notes', 'Experiment ID', 'Researcher', 'Date'],
    'Value': ['This is my experiment description.', 'EXP-2024-001', 'Jane Doe', '2024-12-01']
})

df = pd.DataFrame({
    'Control': [1.2, 2.3, 3.1],
    'Treatment': [2.5, 3.8, 4.2]
})

write_pzfx(df, 'with_notes.pzfx', notes=notes_df)
print("Written with notes. Check 'Project Info' in Prism.")

## 8. Write with subcolumns (replicates)

Use `subcolumns` to specify how many replicates per group, and `subcolumn_suffix` to group columns by a regex pattern.

In [ ]:
# Example: 3 replicates per condition
# Columns named Control_1, Control_2, Control_3 will be grouped as "Control"
df_reps = pd.DataFrame({
    'Control_1': [1.1, 2.1, 3.1],
    'Control_2': [1.2, 2.2, 3.2],
    'Control_3': [1.3, 2.3, 3.3],
    'Treatment_1': [2.1, 3.1, 4.1],
    'Treatment_2': [2.2, 3.2, 4.2],
    'Treatment_3': [2.3, 3.3, 4.3]
})

# subcolumns=3 means 3 replicates
# subcolumn_suffix removes "_1", "_2", "_3" to group columns
write_pzfx(df_reps, 'with_replicates.pzfx', subcolumns=3, subcolumn_suffix='_[0-9]+$')
print("Written with 3 replicates per group")

# Verify structure
print("\nTables:", pzfx_tables('with_replicates.pzfx'))
df_back = read_pzfx('with_replicates.pzfx')
df_back

## 9. Write SDN format (Mean ± SD with N)

Use `subcolumns="SDN"` for pre-calculated summary statistics.

In [ ]:
# SDN format: columns for Mean, SD, and N
# Suffix pattern groups them together
df_sdn = pd.DataFrame({
    'Control_MEAN': [1.5, 2.5, 3.5],
    'Control_SD': [0.2, 0.3, 0.4],
    'Control_N': [5, 5, 5],
    'Treatment_MEAN': [2.5, 3.5, 4.5],
    'Treatment_SD': [0.3, 0.4, 0.5],
    'Treatment_N': [5, 5, 5]
})

write_pzfx(df_sdn, 'sdn_format.pzfx', subcolumns='SDN', subcolumn_suffix='_(MEAN|SD|N)$')
print("Written in SDN format")

## 10. Mark excluded values with asterisk

When writing, values ending with `*` are marked as excluded in Prism.

In [ ]:
# Values ending with * are marked as excluded in Prism
df_excluded = pd.DataFrame({
    'Control': ['1.2', '2.3*', '3.1'],  # 2.3 will be excluded
    'Treatment': ['2.5', '3.8', '4.2*']  # 4.2 will be excluded
})

write_pzfx(df_excluded, 'with_exclusions.pzfx')
print("Written with excluded values")

# Read back with different strike_action to see the effect
print("\nReading back with strike_action='star':")
read_pzfx('with_exclusions.pzfx', strike_action='star')